## NOTES

#### RESOURCES

In [ ]:
https://brennan.io/2016/03/02/logging-in-with-requests/
https://kazuar.github.io/scraping-tutorial/

In [3]:
import json
import pandas as pd
import csv
import datetime
import collections
import re
import copy

from pprint import pprint

from bs4 import BeautifulSoup, NavigableString, Tag
import lxml.html
import requests
from string import capwords
from titlecase import titlecase

pprint('all done')

'all done'


#### COOKIES & LOGIN

In [ ]:
NOT NECCESSARY B/C THE FORUM LIST DOESN'T REQIURE LOGIN TO ACCESS
However, see Prison Talk Scraper Login Notebook for process.

In [ ]:
domain = 'www.prisontalk.com'

headers =   {
                'Host': domain,
                'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.7; rv:44.0) Gecko/20100101 Firefox/44.0',
                'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
                'Accept-Language': 'en-US,en;q=0.5',
                'Accept-Encoding': 'gzip, deflate',
                'DNT': '1',
                'Connection': 'keep-alive'
            }

In [ ]:
username = 'YOUR USERNAME'
password = 'YOUR PASSWORD'

login_form = {
                'vb_login_username': username, 
                'vb_login_password': password,
                'do': 'login',
                'url': '/forums/memberlist.php', 
                'vb_login_md5password': '',
                'vb_login_md5password_utf': '',
                's': '',
                'securitytoken': 'guest'
             }

# login_form

In [ ]:
prison_talk = BeautifulSoup(response.content, 'lxml')

In [ ]:
login_url = 'http://www.prisontalk.com/forums/login.php?do=login'

s.post(login_url, data=form)

s = requests.session()
response = s.get(login_url, headers=headers)
print response.status_code

In [ ]:
login_form = get_login_form(login_page, 'YOUR USERNAME', 'YOUR PASSWORD')
login_form
logged_in = s.post(login_url, data=login_form, headers=headers)
logged_in.status_code

In [ ]:
success_page = BeautifulSoup(logged_in.content)
bool(success_page.find_all(string=re.compile('Redirecting')))

#### REQUEST WEBPAGE AND SAVE TO FILE

In [18]:
forum = 38
order = 'DESC'
sort = 'posts'
pages_per = '100'
page = 1

forum_url = 'http://prisontalk.com/forums/forumdisplay.php?'+ \
                                            'f='+str(forum)+ \
                                            '&order='+order+ \
                                            '&sort='+sort+ \
                                            '&pp='+pages_per+ \
                                            '&page='+str(page)
forum_url

'http://prisontalk.com/forums/forumdisplay.php?f=38&order=DESC&sort=posts&pp=100&page=1'

In [20]:
s = requests.session()
response = s.get(forum_url)

print response.status_code
    
if response.status_code == 200:

    thread_list = BeautifulSoup(response.content, 'lxml')
    
    with open("examples/thread_list.html", "w") as file:
        file.write(str(thread_list))

200


#### LOAD WEBPAGE FROM FILE & REMOVE STRING LITERALS (THEY'RE THEY WORST)

In [59]:
threads = 'thread_list'
threads_w_subforums = 'thread_list_w_sub_forums'

html_page = open('examples/' + threads_w_subforums + '.html', 'r').read()
thread_page = BeautifulSoup(html_page, 'html.parser')
thread_page = str(thread_page)
no_literals = thread_page.replace('\n', '').replace('\t', '').replace('\r', '')


thread_page = BeautifulSoup(no_literals, 'html.parser')#skip u'html PUBLIC "-//W3C//

In [45]:
type(thread_page)

bs4.BeautifulSoup

#### EXTRACT NAV FOR NUMBER OF PAGES

In [108]:
# NOTE: THIS DOES NOT ASSUME EVERY THREAD PAGE WILL HAVE >100 THREADS

def get_num_thread_pages(el):
    
    try:
    
        # THERE IS A TOP AND BOTTOM NAV SO WE'LL JUST TAKE THE TOP
        nav = el.find_all('div', {'class':'pagenav'})[0]
        page_x_of_y = nav.find_all('td', {'class':'vbmenu_control'})[0]
        num_pages = page_x_of_y.text.split(' ')[-1]
        
    except IndexError as er:
        
        num_pages = 1
    
    return num_pages

get_num_thread_pages(thread_page)  #263

u'444'

#### EXTRACT ALL THREADS, THEN REMOVE 'DIVIDER'

In [61]:
def get_threads(el):
    
    thread = copy.copy(el)
    
    posts = thread.find_all(id=re.compile('threadbits_forum'))
    
    # NEED TO REMOVE DIVIDER BETWEEN STICKY POSTS AND REGULAR
    posts[0].find_all('td', {'class' : 'thead'})[0].parent.decompose()
    
    return posts[0]

threads = get_threads(thread_page)
len(threads) # 103

35

#### EXTRACT INFO FROM LISTING OF POSTS

In [62]:
def clean_threads(threads):
    
    data = []
    
    idx = 0

    for thread in threads:
        
        cols = thread.find_all('td')
        
        cols = [cols[2], cols[4], cols[5]]
            
        data.append(cols)

        idx += 1
    
    return data

cleaned_threads = clean_threads(threads)
len(cleaned_threads)

35

In [105]:
def get_thread_data(cleaned):
    
    rows = []
    
    for el in cleaned:
        
                
        thread_id = get_thread_id(el)
        thread_title = get_thread_title(el)
        
        try:

            num_thread_replies = get_thread_replies(el)
            num_thread_views = get_thread_views(el)

            row = [thread_id, thread_title, num_thread_replies, num_thread_views]

            rows.append(row)

    #         print 'ID:', thread_id
    #         print 'TITLE:', thread_title
    #         print 'REPLIES:', num_thread_replies
    #         print 'VIEWS:', num_thread_views
    #         print ''
    
        except Exception as e:
            
            print thread_id + ':', thread_title
        
    return rows

# # [[u'708682', u'LASO Chit Chat 2018', u'47', u'307'],
# #  [u'548317', u'***Thread Titles:  All members please read***', u'2', u'453']]
# data = get_thread_data(cleaned_threads)  
# print len(data)
# data[10:13]

In [64]:
def get_thread_id(el):
    
    # u'showthread.php?s=59e...01&t=32294'
    return el[0].a['href'].split('=')[-1]

get_thread_id(cleaned_threads[0]) # u'32294'

u'708682'

In [95]:
def get_thread_title(el):
    
    text = el[0].find_all(id=re.compile('thread_title'))[0].text
    # <a href="show... id="thread_title_708682"...">LASO Chit Chat 2018</a>
    return text

get_thread_title(cleaned_threads[2]) # u'Some tips on prison "etiquette".'

u'Some tips on prison "etiquette".'

In [96]:
def get_thread_replies(el):
    # <td align='...'><a href="..." onclick="...">540</a></td>
    return el[1].a.text

get_thread_replies(cleaned_threads[0]) # u'540'

u'47'

In [97]:
def get_thread_views(el):
    # <td align="center" class="alt2">92,824</td>
    return el[2].text

get_thread_views(cleaned_threads[0]) # u'92,824'

u'307'

#### WRITE TO CSV (IN THE FUTURE CHANGE TO DICTWRITER)

In [385]:
fieldnames = ['number', 'name', 'threads', 'posts', 'description']

with open('data/forum_list.csv', 'w') as csv_file:
    
    writer = csv.writer(csv_file, delimiter=',')
    writer.writerow(fieldnames)
    
    for forum in forums:
        writer.writerow(forum)